# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

## Setup

In [16]:
import numpy as np
import keras
from keras import layers
from keras.callbacks import ModelCheckpoint

## Prepare the data

In [17]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Build the model

In [18]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1600)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │          16,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

## Train the model

In [19]:
batch_size = 128
epochs = 15

checkpoint = ModelCheckpoint(
    filepath="model_checkpoint_{epoch:02d}.keras",  # Include epoch number in filename
    monitor="val_accuracy",
    save_best_only=False,             # Save model at each epoch, not just best
    mode="max",
    verbose=1
)

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model with the checkpoint

In [21]:
import os
from tensorflow import keras

# Find latest checkpoint file
checkpoint_files = [f for f in os.listdir() if f.startswith("model_checkpoint_")]
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    initial_epoch = int(latest_checkpoint[17:19])  # Extract epoch number from filename
    model = keras.models.load_model(latest_checkpoint)  # Load the model
    print(f"Resuming training from epoch {initial_epoch}")
else:
    initial_epoch = 0  # Start from epoch 0 if no checkpoint found

# ... (rest of the code)

model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    initial_epoch=initial_epoch,  # Specify starting epoch for training
    validation_split=0.1,
    callbacks=[checkpoint]
)

Resuming training from epoch 2
Epoch 3/15
422/422 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9741 - loss: 0.0874
Epoch 3: saving model to model_checkpoint_03.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9741 - loss: 0.0874 - val_accuracy: 0.9882 - val_loss: 0.0477
Epoch 4/15
421/422 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9791 - loss: 0.0676
Epoch 4: saving model to model_checkpoint_04.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9791 - loss: 0.0676 - val_accuracy: 0.9860 - val_loss: 0.0428
Epoch 5/15
412/422 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9798 - loss: 0.0633
Epoch 5: saving model to model_checkpoint_05.keras
422/422 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9798 - loss: 0.0633 - val_accuracy: 0.9885 - val_loss: 0.0405
Epoch 6/15
408/422 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9825 - loss: 0.0572

KeyboardInterrupt: 

## Evaluate the trained model

In [12]:
# Save the model with the correct file extension
model.save("model_checkpoint.h5")

# Evaluate the trained model
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# (Optional) Download the saved model if using Colab
from google.colab import files
files.download("model_checkpoint.h5")

Test loss: 0.023758163675665855
Test accuracy: 0.9934999942779541


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
from tensorflow import keras

# Carga el modelo
modelo_cargado = keras.models.load_model("model_checkpoint.h5")

# Compila el modelo nuevamente
modelo_cargado.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Ahora puedes evaluar el modelo sin la advertencia
score = modelo_cargado.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.023758163675665855
Test accuracy: 0.9934999942779541
